In [1]:
# Adrian Marinovich
# Springboard - Data Science Career Track
# Inferential Statistics Exercise 2
# October 2, 2018

# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

In [3]:
# Setting max columns for viewing:
#pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 100)

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# Check data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [6]:
data.describe()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,h,l,call,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind
count,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4784.000000,4870.000000,4870.000000,1672.000000,1722.000000,608.000000,658.000000,3102.000000,1918.000000,1918.000000,1908.000000,1918.000000,1918.000000,1918.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000
mean,3.618480,3.661396,7.842916,0.052772,0.411499,0.097125,0.448049,215.637782,3.481520,0.559548,0.479261,0.820534,0.328747,0.502259,0.497741,0.080493,651.777832,0.310831,0.542772,10.147275,0.185674,0.213816,9.550801,0.719507,0.291170,587.686035,2287.051025,196.050522,755.416992,0.114765,0.079096,0.843764,10.655662,0.101692,0.333872,10.031516,0.152156,0.077207,0.332854,0.118686,0.151129,0.167967,0.787269,0.435318,0.124846,0.106776,0.437166,0.072690,0.082957,0.030390,0.085010,0.213963,0.267762,0.154825,0.165092
std,0.714997,1.219126,5.044612,0.223601,0.492156,0.296159,0.497345,148.127551,2.038036,0.496492,0.499621,0.383782,0.469806,0.500051,0.500051,0.272079,388.690582,0.332473,0.329467,0.345780,0.081747,0.169305,0.557097,0.449287,0.454347,2907.629395,8902.843750,896.510864,1665.165039,0.318791,0.149742,0.182991,0.441931,0.071293,0.192012,0.567816,0.359208,0.266945,0.471274,0.323461,0.358204,0.373869,0.409275,0.495846,0.330582,0.308866,0.496083,0.259649,0.275854,0.171677,0.278932,0.410141,0.442847,0.361773,0.371308
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.004814,8.841738,0.000000,0.030847,8.507345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005500,9.170247,0.000000,0.030847,8.662505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,306.250000,0.045275,0.252164,9.965053,0.139711,0.092559,9.220489,0.000000,0.000000,12.975000,98.000000,13.000000,97.000000,0.000000,0.007125,0.824140,10.448976,0.047958,0.201971,9.691531,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,267.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,647.000000,0.159950,0.571833,10.144078,0.190751,0.145053,9.438432,1.000000,0.000000,33.350000,220.000000,34.900000,200.000000,0.000000,0.017404,0.900727,10.666441,0.087009,0.288414,9.914428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,0.000000,1.000000,0.000000,1.000000,313.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,979.750000,0.516854,0.873805,10.342871,0.238196,0.284315,9.668208,1.000000,1.000000,133.099998,700.000000,86.69999

In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

Q1. _What test is appropriate for this problem? Does CLT apply?_

The chi-square test is appropriate, since both 'race' and 'call' are binary categorical variables, and the test is therefore a comparison of two proportions. The CLT does not apply.

Q2. _What are the null and alternate hypotheses?_

For chi-square testing, the null hypothesis is that 'race' and 'call' are independent, while the alternative hypothesis is that 'race' and 'call' are associated with each other.

Q3. _Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches._



Note: 

The term "margin of error" is used inconsistently, leads to confusion in the biomedical literature that can be manipulated to cherry pick favorable interpretations (e.g., one vs. two standard deviations), and ultimately puts the onus of calculation on the reader. I believe an explicit confidence interval is the preferrable way to communicate the concept. For the purposes of these exercises only, I will designate half of the 95% confidence interval as the margin or error, and insert this value following "+/-" at the end of the appropriate discussions.

In [8]:
# Q3. (continued...)
# Using a frequentist approach:

# Make contingency table
contingency = pd.crosstab(data['race'], data['call'])
contingency

call,0.0,1.0
race,,
b,2278,157
w,2200,235


In [9]:
# Q3. (continued...)
# Calculate callback frequencies directly from data

# Import the following to specify float division
from __future__ import division

bcall = len(data[(data.race=='b') & (data.call==1)])/len(data[(data.race=='b')])
wcall = len(data[(data.race=='w') & (data.call==1)])/len(data[(data.race=='w')])

print('Callback frequency among black-sounding names:', round(bcall,4), 
      '\nCallback frequency among white-sounding names:', round(wcall,4))

Callback frequency among black-sounding names: 0.0645 
Callback frequency among white-sounding names: 0.0965


In [10]:
# Q3. (continued...)
# Calculate callback odds ratio
odds_ratio = wcall/bcall
print("white callback : black callback Odds Ratio...", round(odds_ratio,3))

white callback : black callback Odds Ratio... 1.497


In [11]:
# Q3. (continued...)
# Perform chi-square on contingency array (outputs 
#  chi-square statistic, p value, degrees of freedom, expected frequencies array)
chisq, pvalue, dof, exp_arr = scipy.stats.chi2_contingency(contingency)

In [12]:
pvalue

4.997578389963255e-05

In [25]:
# Q3. (continued...)
# Compute 95% confidence interval of odds ratio

# Calculate log odds ratio and its standard error
LOR = np.log(odds_ratio)
SE = np.sqrt(np.sum(1/contingency.astype(np.float64)))

# 95% confidence intervals
lower = np.exp(LOR - 1.96*SE[1])
upper = np.exp(LOR + 1.96*SE[1])

print('The Odds Ratio 95% confidence interval for white:black callback is:', round(lower,3), '-', round(upper,3))

The Odds Ratio 95% confidence interval for white:black callback is: 1.223 - 1.832


Q3. (continued...)

With a p value of 0.00005, we can say that race/name is significantly associated with callback frequency, and in particular, white race/name is associated with a 1.5-fold increased callback frequency compared to black race/name (95% confidence interval: 1.2-1.8).

(White:Black callback odds ratio with margin of error: 1.5 +/- 0.3)

In [14]:
# Q3. (continued...)
# Using a bootstrap approach:

# Define binomial random generating function that outputs 
#  proportion of positive (1) trials
def binomial_prob(trials, prob, reps):
    return np.sum(np.random.binomial(trials, prob, reps))/reps

In [15]:
# Q3. (continued...)

# Running replicates using black race/name callback probability with 
#  Bernoulli (single trial) specified in the binomial
  
b_replicates = np.empty(10000)

for i in range(10000):
    b_replicates[i] = binomial_prob(1, bcall, 20000)

conf_int = np.percentile(b_replicates, [2.5, 97.5])
print('Bootstrap 95% CI for black-sounding names:', conf_int)

Bootstrap 95% CI for black-sounding names: [0.0611 0.0678]


In [16]:
# Q3. (continued...)

# Running replicates using white race/name callback probability with 
#  Bernoulli (single trial) specified in the binomial

w_replicates = np.empty(10000)

for i in range(10000):
    w_replicates[i] = binomial_prob(1, wcall, 20000)

conf_int = np.percentile(w_replicates, [2.5, 97.5])
print('Bootstrap 95% CI for white-sounding names:', conf_int)

Bootstrap 95% CI for white-sounding names: [0.0924 0.1006]


Q3. (continued...)

Using the bootstrap approach to generate 95% confidence intervals (95% CI) using the Bernoulli (single trial) case of the binomial distribution, with 20000 random binomial repetitions replicated 10000 times for the given probabilities for black- and white-sounding names, we see that the callback percentage for resumes with black-sounding names has a 95% CI of 6.1-6.8%, while for resumes with white-sounding names it is 9.2-10.1%. Since these are non-overlapping confidence intervals, we can say that the callback proportion for white race/name is significantly higher than for black race/name.

(Callback percentage for black race/name resumes with margin of error: 6.45% +/- 0.34)

(Callback percentage for white race/name resumes with margin of error: 9.65% +/- 0.41)

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Q4. _Write a story describing the statistical significance in the context or the original problem._

As the above analysis shows, resumes with black-sounding names are significantly less likely to receive a call back than resumes with white-sounding names (p=0.00005). The callback frequency for resumes with black-sounding names is 6.45%, while the callback frequency for white-sounding names is 9.65%. This means that resumes with white-sounding names are about 50% more likely to receive a callback, when all else is equal.

Q5. _Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?_

The dataset is the result of a randomized experiment, wherein resumes were randomly assigned either a black-sounding or white-sounding name (coded as 'race') before being sent out, with call back being the planned outcome variable (coded as 'call'). This means that the values for other variables measured during the study, such as 'education', 'honors', etc. are control variables that should have no association with 'race', and therefore should not affect the effect size seen for race.

We can check the randomization protocol by checking that 'race' does indeed have no association with a selection of some of the more salient control variables:

In [17]:
# Q5. (continued...)
data.groupby('race')['ad','education','ofjobs','yearsexp','honors','volunteer','military',
                     'empholes','workinschool','computerskills','specialskills','sex',
                     'manager','supervisor','secretary'].mean()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,workinschool,computerskills,specialskills,manager,supervisor,secretary
race,,,,,,,,,,,,,
b,3.616016,3.658316,7.829569,0.051335,0.414374,0.101848,0.445996,0.560986,0.832444,0.327310,0.151951,0.077207,0.333060
w,3.620945,3.664476,7.856263,0.054209,0.408624,0.092402,0.450103,0.558111,0.808624,0.330185,0.152361,0.077207,0.332649


Only slight variations in these variables are seen across the 'race' category, indicating that randomization was successful.

Once we accept that race has an independent effect, we then can ask whether there are _other_ factors that have their own indepedent effects on callback frequency. Looking at the same control variables as above, we can perform the same analysis, except this time grouping by 'call' instead of 'race':

In [18]:
# Q5. (continued...)
data.groupby('call')['ad','education','ofjobs','yearsexp','honors','volunteer','military',
                     'empholes','workinschool','computerskills','specialskills','sex',
                     'manager','supervisor','secretary'].mean()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,workinschool,computerskills,specialskills,manager,supervisor,secretary
call,,,,,,,,,,,,,
0.0,3.619696,3.660563,7.751228,0.048013,0.410451,0.098928,0.437472,0.563644,0.823805,0.313310,0.154310,0.078160,0.332291
1.0,3.604592,3.670918,8.890306,0.107143,0.423469,0.076531,0.568878,0.512755,0.783163,0.505102,0.127551,0.066327,0.339286


Now we see that a number of control variables exhibit substantial differences in their means when comparing call back ('call' = 1) to no call back ('call' = 0). In particular, we note that 'honors' shows an impressive difference. We look at this variable more closely in a contingency table and a chi-square test:

In [19]:
# Q5. (continued...)
contingency_h = pd.crosstab(data['honors'], data['call'])
contingency_h

call,0.0,1.0
honors,,
0,4263,350
1,215,42


In [20]:
# Q5. (continued...)
from __future__ import division

h0call = len(data[(data.honors==0) & (data.call==1)])/len(data[(data.honors==0)])
h1call = len(data[(data.honors==1) & (data.call==1)])/len(data[(data.honors==1)])

print('Callback frequency with no honors (0):', round(h0call,4), 
      '\nCallback frequency with honors (1):', round(h1call,4))

Callback frequency with no honors (0): 0.0759 
Callback frequency with honors (1): 0.1634


In [21]:
# Q5. (continued...)
odds_ratio_h = h1call/h0call
print("honors callback : no honors callback Odds Ratio...", round(odds_ratio_h,3))

honors callback : no honors callback Odds Ratio... 2.154


In [23]:
# Q5. (continued...)
# Perform chi-square on contingency array (outputs 
#  chi-square statistic, p value, degrees of freedom, expected frequencies array)
chisq_h, pvalue_h, dof_h, exp_arr_h = scipy.stats.chi2_contingency(contingency_h)
pvalue_h

9.422073783062283e-07

In [24]:
# Q5. (continued...)
# Compute 95% confidence interval of honors callback odds ratio

# Calculate log odds ratio and its standard error
LOR_h = np.log(odds_ratio_h)
SE_h = np.sqrt(np.sum(1/contingency_h.astype(np.float64)))

# 95% confidence intervals
lower_h = np.exp(LOR_h - 1.96*SE_h[1])
upper_h = np.exp(LOR_h + 1.96*SE_h[1])

print('The Odds Ratio 95% confidence interval for honors:no honors callback is:', round(lower_h,3), '-', round(upper_h,3))

The Odds Ratio 95% confidence interval for honors:no honors callback is: 1.564 - 2.966


Q5. (continued...)

We see that the presence of honors has a substantial effect size, associated with about twice the callback rate compared to no honors. This effect is clearly significantly associated with callback (p=0.0000009).

While race/name is an important independent factor in callback success, it is not the only important factor, as for example, the presence of honors appears to have a significant independent effect on callback than race/name. It is worth noting, however, that the confidence intervals for these two variables overlap, indicating that there is no significant difference between their effect sizes.